* [#4](https://github.com/salgo60/SCB-Wikidata/issues/4)
* denna [SCB4_linkroot_v2.ipynb](https://github.com/salgo60/SCB-Wikidata/blob/main/notebook/SCB4_linkroot_v2.ipynb)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-11-29 23:43:47


In [2]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-11-29 23:43:47


In [3]:
SCRIPT_NAME = "SCB4_linkroot_v2.ipynb"
SCRIPT_URL = (
    "https://github.com/salgo60/SCB-Wikidata/"
    "blob/master/notebook/SCB4_linkroot_v2.ipynb"
) 


In [4]:
def read_domains(file_path):
    print(f"[DEBUG] Reading domains from: {file_path}")
    df = pd.read_csv(file_path, header=0)   # <- skip header row
    domains_list = df.iloc[:, 0].dropna().unique().tolist()
    print(f"[DEBUG] Found {len(domains_list)} domains.")
    return domains_list


In [5]:
import requests

def fetch_sitematrix_df():
    url = "https://meta.wikimedia.org/w/api.php"
    params = {
        "action": "sitematrix",
        "format": "json"
    }
    headers = {
        "User-Agent": "salgo60-language-fetcher/1.0 (salgo60@msn.com)"
    }

    print("[DEBUG] Fetching sitematrix…")
    r = requests.get(url, params=params, headers=headers)
    r.raise_for_status()

    if "application/json" not in r.headers.get("Content-Type", ""):
        raise ValueError("Server returned non-JSON response")

    data = r.json()["sitematrix"]

    rows = []

    # --- language-specific sites ---
    for key, lang_block in data.items():
        if not key.isdigit():
            continue  # skip "count", "specials"

        lang_code = lang_block.get("code")
        lang_name = lang_block.get("name")

        for site in lang_block.get("site", []):
            rows.append({
                "lang_code": lang_code,
                "lang_name": lang_name,
                "project": site.get("project"),
                "url": site.get("url"),
                "dbname": site.get("dbname"),
                "site_name": site.get("sitename"),
                "closed": site.get("closed", False)
            })

    # --- special wikis (Wikidata, Commons, Meta, etc.) ---
    for site in data.get("specials", []):
        rows.append({
            "lang_code": "special",
            "lang_name": "special",
            "project": site.get("project"),
            "url": site.get("url"),
            "dbname": site.get("dbname"),
            "site_name": site.get("sitename"),
            "closed": site.get("closed", False)
        })

    return pd.DataFrame(rows)


In [6]:
import requests
import pandas as pd


HEADERS = {
    "User-Agent": "salgo60-language-fetcher/2.0 (https://github.com/salgo60)"
}


df_lang_fetch = fetch_sitematrix_df()
df_lang_fetch["closed"] = df_lang_fetch["closed"].fillna(False).astype(bool)

df_lang_fetch.head()



[DEBUG] Fetching sitematrix…


,lang_code,lang_name,project,url,dbname,site_name,closed
0,aa,Qafár af,None,https://aa.wikipedia.org,aawiki,Wikipedia,False
1,aa,Qafár af,None,https://aa.wiktionary.org,aawiktionary,Wiktionary,False
2,aa,Qafár af,None,https://aa.wikibooks.org,aawikibooks,Wikibooks,False
3,ab,аԥсшәа,None,https://ab.wikipedia.org,abwiki,Авикипедиа,False
4,ab,аԥсшәа,None,https://ab.wiktionary.org,abwiktionary,Wiktionary,False


In [7]:
df_lang_fetch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063 entries, 0 to 1062
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lang_code  1063 non-null   object
 1   lang_name  1060 non-null   object
 2   project    0 non-null      object
 3   url        1063 non-null   object
 4   dbname     1063 non-null   object
 5   site_name  1063 non-null   object
 6   closed     1063 non-null   bool  
dtypes: bool(1), object(6)
memory usage: 51.0+ KB


In [8]:
df_wiki = df_lang_fetch[df_lang_fetch["site_name"] == "Wikipedia"] 
df_wiki.info()

<class 'pandas.core.frame.DataFrame'>
Index: 207 entries, 0 to 1061
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lang_code  207 non-null    object
 1   lang_name  206 non-null    object
 2   project    0 non-null      object
 3   url        207 non-null    object
 4   dbname     207 non-null    object
 5   site_name  207 non-null    object
 6   closed     207 non-null    bool  
dtypes: bool(1), object(6)
memory usage: 11.5+ KB


In [9]:
df_wiki.head()

,lang_code,lang_name,project,url,dbname,site_name,closed
0,aa,Qafár af,None,https://aa.wikipedia.org,aawiki,Wikipedia,False
5,ace,Acèh,None,https://ace.wikipedia.org,acewiki,Wikipedia,False
7,af,Afrikaans,None,https://af.wikipedia.org,afwiki,Wikipedia,False
11,ak,None,None,https://ak.wikipedia.org,akwiki,Wikipedia,False
18,ami,Pangcah,None,https://ami.wikipedia.org,amiwiki,Wikipedia,False


In [10]:
df_lang_fetch["closed"].value_counts()

closed
False    1063
Name: count, dtype: int64

In [11]:
len(df_wiki)

207

In [12]:
# Finns även specials
df_wiki["closed"].value_counts()

closed
False    207
Name: count, dtype: int64

In [13]:
df_wiki = df_lang_fetch[df_lang_fetch["site_name"] == "Wikipedia"] 
df_wiki.info()

<class 'pandas.core.frame.DataFrame'>
Index: 207 entries, 0 to 1061
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lang_code  207 non-null    object
 1   lang_name  206 non-null    object
 2   project    0 non-null      object
 3   url        207 non-null    object
 4   dbname     207 non-null    object
 5   site_name  207 non-null    object
 6   closed     207 non-null    bool  
dtypes: bool(1), object(6)
memory usage: 11.5+ KB


In [14]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)



Current Working Directory: /Users/salgo/Documents/GitHub/SCB-Wikidata/notebook


In [15]:
# -----------------------------------------------------------
# Fetch exturlusage entries for one lang/domain
# -----------------------------------------------------------
def fetch_exturlusage(lang, domain):
    base = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "exturlusage",
        "euquery": domain,
        "eulimit": "max"
    }
    while True:
        r = session.get(base, params=params, timeout=10)
        try:
            data = r.json()
        except ValueError:
            print(f"[WARN] {lang}: JSON decode failed")
            break

        for item in data.get("query", {}).get("exturlusage", []):
            yield {
                "lang": lang,
                "page_title": item.get("title"),
                "url": item.get("url"),
                "wiki_link": f"https://{lang}.wikipedia.org/wiki/{item.get('title').replace(' ', '_')}"
            }

        if "continue" not in data:
            break
        params.update(data["continue"])
        time.sleep(0.3)

In [16]:
import os
import time
import random
import requests
import pandas as pd
from urllib.parse import urlparse
from tqdm.notebook import tqdm
file_path_domain = "sources/domains.csv"
domains = read_domains(file_path_domain)
print(domains)


[DEBUG] Reading domains from: sources/domains.csv
[DEBUG] Found 1 domains.
['scb.se']


In [17]:

# -------------------------
# Session & helpers
# -------------------------
session = requests.Session()
session.headers.update({"User-Agent": "SCB-LinkAudit/1.0 (https://www.scb.se/)"})

# we need some filtering  
df_wiki_lang = df_wiki[df_wiki["lang_code"] != "special"] 

print("Antal Språk:",len(df_wiki_lang ))
results = []
for _, row in df_wiki_lang.iterrows():
    lang = row["lang_code"]
    url  = row["url"]
    lang_name = row["lang_name"]
    before = len(results)
    #print(lang, url, lang_name,domains)
    for entry in fetch_exturlusage(lang, domains):
        results.append(entry)     
    after = len(results) 
    links = after-before
    print(lang, url, lang_name," - ", links)
    


Antal Språk: 184
aa https://aa.wikipedia.org Qafár af  -  0
ace https://ace.wikipedia.org Acèh  -  0
af https://af.wikipedia.org Afrikaans  -  53
ak https://ak.wikipedia.org None  -  0
ami https://ami.wikipedia.org Pangcah  -  0
an https://an.wikipedia.org aragonés  -  0
ast https://ast.wikipedia.org asturianu  -  7
av https://av.wikipedia.org авар  -  2
avk https://avk.wikipedia.org Kotava  -  0
ay https://ay.wikipedia.org Aymar aru  -  0
bar https://bar.wikipedia.org Boarisch  -  102
bbc https://bbc.wikipedia.org Batak Toba  -  0
bcl https://bcl.wikipedia.org Bikol Central  -  5
bi https://bi.wikipedia.org Bislama  -  0
bm https://bm.wikipedia.org bamanankan  -  0
bo https://bo.wikipedia.org བོད་ཡིག  -  0
br https://br.wikipedia.org brezhoneg  -  1
bs https://bs.wikipedia.org bosanski  -  20
btm https://btm.wikipedia.org Batak Mandailing  -  0
bug https://bug.wikipedia.org Basa Ugi  -  0
bxr https://bxr.wikipedia.org буряад  -  9
cbk-zam https://cbk-zam.wikipedia.org Chavacano de Zam

ReadTimeout: HTTPSConnectionPool(host='sv.wikipedia.org', port=443): Read timed out. (read timeout=10)

In [18]:
domains

['scb.se']

In [19]:
df_wiki.head()

,lang_code,lang_name,project,url,dbname,site_name,closed
0,aa,Qafár af,None,https://aa.wikipedia.org,aawiki,Wikipedia,False
5,ace,Acèh,None,https://ace.wikipedia.org,acewiki,Wikipedia,False
7,af,Afrikaans,None,https://af.wikipedia.org,afwiki,Wikipedia,False
11,ak,None,None,https://ak.wikipedia.org,akwiki,Wikipedia,False
18,ami,Pangcah,None,https://ami.wikipedia.org,amiwiki,Wikipedia,False


In [20]:
df_SCB = pd.DataFrame(results)

In [21]:
len(df_SCB )

94248

In [22]:
df_SCB

,lang,page_title,url,wiki_link
0,af,Noorweërs,http://www.scb.se/statistik/_publikationer/BE0...,https://af.wikipedia.org/wiki/Noorweërs
1,af,Sweeds,http://www.scb.se/statistik/_publikationer/BE0...,https://af.wikipedia.org/wiki/Sweeds
2,af,Perse,http://www.scb.se/Statistik/BE/BE0101/2011A01B...,https://af.wikipedia.org/wiki/Perse
3,af,Gebruikerbespreking:Morne/argief 3,http://www.scb.se/Statistik/UF/UF0205/2014L15E...,https://af.wikipedia.org/wiki/Gebruikerbesprek...
4,af,Lys van die digs bevolkte munisipaliteite in d...,http://www.scb.se/sv_/Hitta-statistik/Statisti...,https://af.wikipedia.org/wiki/Lys_van_die_digs...
...,...,...,...,...
94243,sv,Grebbestad,http://geodata.scb.se/reginawebmap/main/webapp...,https://sv.wikipedia.org/wiki/Grebbestad
94244,sv,Glemmingebro,http://www.scb.se/Statistik/MI/MI0810/2010A01Z...,https://sv.wikipedia.org/wiki/Glemmingebro
94245,sv,Glemmingebro,http://geodata.scb.se/reginawebmap/main/webapp...,https://sv.wikipedia.org/wiki/Glemmingebro
94246,sv,Gideå,http://www.scb.se/Statistik/MI/MI0810/2010A01Z...,https://sv.wikipedia.org/wiki/Gideå


In [23]:
import pandas as pd

# --- Stats ---
total_links = len(df_SCB)
total_unique_links = df_SCB['url'].nunique()
num_languages = df_SCB['lang'].nunique()
langs_sorted = df_SCB['lang'].value_counts()

print("Total links:", total_links)
print("Total unique links:", total_unique_links)
print("Number of languages:", num_languages)
print("\nLanguages with most links:")
print(langs_sorted.to_string())


Total links: 94248
Total unique links: 17283
Number of languages: 64

Languages with most links:
lang
sv          52000
fi           9315
no           6632
en           6404
pl           2668
de           2223
it           2119
nan          1981
ro           1905
da           1817
simple       1524
es           1455
fo            654
eu            585
sco           457
gl            393
nn            362
nl            352
is            315
id            180
ja            162
lld           152
bar           102
se             79
sq             66
min            60
af             53
frr            48
ms             37
bs             20
ha             17
ilo            13
su             12
ia             10
bxr             9
ast             7
stq             6
so              6
bcl             5
rm              4
kaa             4
gv              4
iba             3
av              2
pap             2
ceb             2
lb              2
ig              2
new             2
mg              

In [24]:
from urllib.parse import urlparse
import pandas as pd

# Extract domains
df_SCB['domain'] = df_SCB['url'].apply(lambda u: urlparse(u).netloc)

# Count links grouped by domain
links_per_domain = df_SCB['domain'].value_counts()

print("Links per domain:")
print(links_per_domain.to_string())


Links per domain:
domain
www.scb.se                        64821
geodata.scb.se                    12506
www.statistikdatabasen.scb.se     11568
share.scb.se                       2419
kommunsiffror.scb.se               1809
www.ssd.scb.se                      521
scb.se                              222
www.sverigeisiffror.scb.se          162
www.gis.scb.se                      106
regina.scb.se                        25
www.myndighetsregistret.scb.se       18
www.h.scb.se                         13
w41.scb.se                           12
www.pubkat.scb.se                     8
www.scb.se:80                         6
w36.scb.se                            5
myndighetsregistret.scb.se            3
www.h5.scb.se                         3
statistikdatabasen.scb.se             2
w42.ssd.scb.se                        2
www.orestat.scb.se                    2
ssd.scb.se                            2
www.pc-axis.scb.se                    2
www..scb.se                           2
api.scb.se     

In [25]:
df_SCB = df_SCB.rename(columns={
    "lang": "Språk",
    "page_title": "Wikipedia-sida",
    "url": "Extern länk",
    "wiki_link": "Wikipedia-länk",
    "domain": "Domän"
})


In [26]:
import pandas as pd

# --- Stats ---
total_links = len(df_SCB)
total_unique_links = df_SCB['Extern länk'].nunique()
num_languages = df_SCB['Språk'].nunique()
langs_sorted = df_SCB['Språk'].value_counts()

print("Total links:", total_links)
print("Total unique links:", total_unique_links)
print("Number of languages:", num_languages)
print("\nLanguages with most links:")
print(langs_sorted.to_string())


Total links: 94248
Total unique links: 17283
Number of languages: 64

Languages with most links:
Språk
sv          52000
fi           9315
no           6632
en           6404
pl           2668
de           2223
it           2119
nan          1981
ro           1905
da           1817
simple       1524
es           1455
fo            654
eu            585
sco           457
gl            393
nn            362
nl            352
is            315
id            180
ja            162
lld           152
bar           102
se             79
sq             66
min            60
af             53
frr            48
ms             37
bs             20
ha             17
ilo            13
su             12
ia             10
bxr             9
ast             7
stq             6
so              6
bcl             5
rm              4
kaa             4
gv              4
iba             3
av              2
pap             2
ceb             2
lb              2
ig              2
new             2
mg             

In [27]:
from datetime import date
import os

# Sätt datum
today = date.today().strftime("%Y_%m_%d")

# Se till att katalogen finns
os.makedirs("results", exist_ok=True)

# Bygg filnamn
outfile = f"results/links_SCB_v2_{today}.csv"

# Exportera
df_SCB.to_csv(outfile, index=False, encoding="utf-8")

print(f"[OK] Exported {len(df_SCB)} rows to {outfile}")


[OK] Exported 94248 rows to results/links_SCB_v2_2025_11_30.csv


In [28]:
from urllib.parse import urlparse
import pandas as pd

# Extract domain
df_SCB['domain'] = df_SCB['Extern länk'].apply(lambda u: urlparse(u).netloc)

# Count links grouped by domain
links_per_domain = df_SCB['domain'].value_counts()

print("Links per domain:")
print(links_per_domain.to_string())


Links per domain:
domain
www.scb.se                        64821
geodata.scb.se                    12506
www.statistikdatabasen.scb.se     11568
share.scb.se                       2419
kommunsiffror.scb.se               1809
www.ssd.scb.se                      521
scb.se                              222
www.sverigeisiffror.scb.se          162
www.gis.scb.se                      106
regina.scb.se                        25
www.myndighetsregistret.scb.se       18
www.h.scb.se                         13
w41.scb.se                           12
www.pubkat.scb.se                     8
www.scb.se:80                         6
w36.scb.se                            5
myndighetsregistret.scb.se            3
www.h5.scb.se                         3
statistikdatabasen.scb.se             2
w42.ssd.scb.se                        2
www.orestat.scb.se                    2
ssd.scb.se                            2
www.pc-axis.scb.se                    2
www..scb.se                           2
api.scb.se     

In [29]:
GITHUB_REPO = "https://github.com/salgo60/Wikidata"


In [30]:
script_name = SCRIPT_NAME

In [31]:
script_name

'SCB4_linkroot_v2.ipynb'

In [32]:
print(df_SCB.columns.tolist())

['Språk', 'Wikipedia-sida', 'Extern länk', 'Wikipedia-länk', 'Domän', 'domain']


In [33]:
# --- Stats ---
total_links = len(df_SCB)
total_unique_links = df_SCB["Extern länk"].nunique()
langs_with_hits = sorted(df_SCB["Språk"].unique())

num_languages_found = len(langs_with_hits)
num_languages_checked = len(df_wiki)        # alla språk som genomsöktes
num_languages_found = df_SCB['Språk'].nunique()

In [38]:
from pathlib import Path
from datetime import date
import pandas as pd

def save_sortable_html_df_SCB(
    df,
    out_dir="results",
    domains=None,
    issue_url="https://github.com/salgo60/SCB-Wikidata/issues/4",
):
    out_dir = Path(out_dir)
    out_dir.mkdir(exist_ok=True)

    today = date.today().strftime("%Y_%m_%d")
    out_path = out_dir / f"links_SCB_v2_{today}.html"
    rerun_ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Metadata
 
    domains = domains or [] 

    # Gör länkar klickbara om de finns
    df = df.copy()  
    if "Wikipedia-länk" in df.columns:
        df["Wikipedia-länk"] = df["Wikipedia-länk"].apply(
            lambda x: f'<a href="{x}" target="_blank">{x}</a>' if pd.notna(x) else ""
        )
    if "Extern länk" in df.columns:
        df["Extern länk"] = df["Extern länk"].apply(
            lambda x: f'<a href="{x}" target="_blank">{x}</a>' if pd.notna(x) else ""
        )

    html_table = df.to_html(
        classes="pivot",
        border=0,
        escape=False,   # viktigt för klickbara länkar
        index=False
    )
   
    css = """
    <style>
      body {
        font-family: Arial, sans-serif;
        margin: 20px;
      }
      table.pivot {
        border-collapse: collapse;
        width: 100%;
        font-size: 12px;
      }
      table.pivot th, table.pivot td {
        border: 1px solid #999;
        padding: 6px 8px;
        text-align: left;
        vertical-align: top;
        white-space: normal;
      }
      table.pivot th {
        cursor: pointer;
        background: #f2f2f2;
      }
      table.pivot th:hover {
        background: #e2e2e2;
      } 
      table.pivot thead th {
          position: sticky;
          top: 0;
          background: #f2f2f2;
          z-index: 2;
        }
        
        /* Sorting indicators */
        table.pivot th::after {
          content: "";
          float: right;
          opacity: 0.4;
        }
        
        table.pivot th.sorted-asc::after {
          content: " ▲";
        }
        
        table.pivot th.sorted-desc::after {
          content: " ▼";
        }
          .meta {
            background: #f8f8f8;
            border: 1px solid #ccc;
            padding: 12px;
            margin-bottom: 20px;
            font-size: 13px;
          }
          .meta h2 {
            margin-top: 0;
          }
          .meta ul {
            margin: 0;
            padding-left: 18px;
          }
    </style>
    """

    js = """
    <script>
    document.addEventListener('DOMContentLoaded', () => {
        document.querySelectorAll("table.pivot th").forEach((header, colIndex) => {
            header.addEventListener("click", () => {
                const table = header.closest("table");
                const tbody = table.querySelector("tbody");
                const rows = Array.from(tbody.querySelectorAll("tr"));
                const asc = !header.classList.contains("sorted-asc");

                rows.sort((a, b) => {
                    const A = a.children[colIndex].innerText.trim();
                    const B = b.children[colIndex].innerText.trim();

                    const numA = parseFloat(A.replace(",", ".")) || null;
                    const numB = parseFloat(B.replace(",", ".")) || null;

                    if (numA !== null && numB !== null) {
                        return asc ? numA - numB : numB - numA;
                    }
                    return asc ? A.localeCompare(B) : B.localeCompare(A);
                });

                table.querySelectorAll("th").forEach(th =>
                    th.classList.remove("sorted-asc","sorted-desc")
                );
                header.classList.add(asc ? "sorted-asc" : "sorted-desc");

                rows.forEach(row => tbody.appendChild(row));
            });
        });
    });
    </script>
    """
    meta_html = f"""
    <div class="meta">
      <h2>Metadata</h2>
    
      <p><strong>Rerun:</strong> {rerun_ts}</p>
      <p><strong>Script:</strong>
         <a href="{SCRIPT_URL}" target="_blank">{SCRIPT_NAME}</a>
      </p>
      <p><strong>Issue:</strong>
         <a href="{issue_url}" target="_blank">{issue_url.split('/')[-1]}</a>
      </p>
    
      <p><strong>Datakällor:</strong></p>
      <div class="meta-indent">
        Wikidata<br>
        MediaWiki API – exturlusage
      </div>
    
      <p><strong>Parametrar:</strong></p>
      <div class="meta-indent">
        Språkfilter: exkluderar "special"
      </div>
    
      <p><strong>Antal språk genomsökta:</strong> {num_languages_checked}</p>
      <p><strong>Antal språk med träffar:</strong> {num_languages_found}</p>
      <p><strong>Wikipedia-språk med träffar:</strong> {", ".join(langs_with_hits)}</p>
      <p><strong>Totala länkar:</strong> {total_links}</p>
      <p><strong>Unika länkar:</strong> {total_unique_links}</p>
    </div>
    """
    
    html = f"""
    <html>
    <head>
      <meta charset="utf-8">
      <title>SCB links in Wikipedia</title>
      {css}
    </head>
    <body>
      <h1>Wikipedia → SCB v2</h1>
      {meta_html}
      <p>Sorterbar tabell. Klicka på kolumnrubriker för sortering.</p>
      {html_table}
      {js}
    </body>
    </html>
    """

    out_path.write_text(html, encoding="utf-8")
    print(f"✅ HTML skapad: {out_path}")


In [39]:

save_sortable_html_df_SCB(df_SCB)


✅ HTML skapad: results/links_SCB_v2_2025_11_30.html


In [ ]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))
